<a href="https://colab.research.google.com/github/ghigoviu/proyecto1/blob/main/Ecout%C3%A9_Exani1_Modulo_construccion_de_versiones_Exani1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Ecouté Exani1
# Módulo de construcción de cuadernillos Exani 1

### Prerequisitos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install python-docx
!pip install docxcompose

import re
import os
import csv
from docx import Document
from docx.shared import Pt
from docx.shared import Inches
from docx.shared import Cm
from docxcompose.composer import Composer
from docx.enum.text import WD_LINE_SPACING

## Definición de clases

### Clase cuadernillo

In [ ]:
class Cuadernillo:
  def __init__(self):
    self.numero_cuadernillo
    self.ccf
    self.controles = []
    self.bloques = []
    self.bloques_piloto = [] # Son las claves del piloto
    self.clave_control
    self.respuestas_control = []
    self.orden
    self.documento

  def set_version_maestra(self, version_maestra):
    self.version_maestra = version_maestra

  def get_version_maestra(self):
    return self.version_maestra

  def set_numero_cuadernillo(self, numero_cuadernillo):
    self.numero_cuadernillo = numero_cuadernillo

  def get_numero_cuadernillo(self):
    return self.numero_cuadernillo

  def set_ccf(self, ccf):
    self.ccf = ccf

  def get_ccf(self):
    return self.ccf

  def set_controles(self, controles):
    self.controles = controles

  def get_controles(self):
    return self.controles

  def set_clave_control(self, clave_control):
    self.clave_control = clave_control

  def get_bloques_piloto(self):
    return self.bloques_piloto

  def set_bloques_piloto(self, bloque1, bloque2):
    self.bloques_piloto = []
    self.bloques_piloto.append(bloque1)
    self.bloques_piloto.append(bloque2)

  def set_respuestas_control(self, respuestas_control):
    self.respuestas_control = respuestas_control

  def get_respuestas_control(self):
    return self.respuestas_control

  def set_orden(self, orden):
    self.orden = orden

  def get_orden(self):
    return self.orden

  def set_documento(self, doc):
    self.documento = doc

  def get_documento(self):
    return self.documento

  def generar_portada(self):
    print("Generando portadas")

  def construir_cuadernillo(self):
    print("Construyendo cuadernillo")

  def agregar_titulos(self):
    print("Agregando titulos")


### Clase Exani1

In [ ]:
class Exani1(Cuadernillo):
  def __init__(self):
    super()
    self.version_ingles = 0
    self.reactivos = []
    tipo_pilotaje = "" # 4 tipos de piloteje según la posición de los bloques piloto y el área que pilotean.

  def get_version_ingles(self):
    return self.version_ingles

  def set_version_ingles(self, version_ingles):
    self.version_ingles = version_ingles

  def get_reactivos(self):
    return self.reactivos

  def set_reactivos(self, reactivos):
    self.reactivos = reactivos


### Clase Ensamblador

In [ ]:
class Ensamblador:
  def __init__(self, csv_doc):
    self.orden1 = []
    self.orden2 = []
    self.maestra = Document()
    self.cuadernillos = []
    self.datos = []
    with open(csv_doc, encoding='ISO-8859-1') as csv_file:
      csv_reader = csv.reader(csv_file, delimiter=',')
      for row in csv_reader:
        self.datos.append(row)

  def get_orden1(self):
    return self.orden1

  def set_cuadernillos(self, cuadernillos):
    self.cuadernillos = cuadernillos

  def get_cuadernillos(self):
    return self.cuadernillos

  def set_orden1(self, orden1):
    self.orden1 = orden1

  def get_maestra(self):
    return self.maestra

  def set_maestra(self, maestra):
    self.maestra = maestra

  def get_orden2(self):
    return self.orden2

  def set_orden2(self, orden2):
    self.orden2 = orden2

  def get_datos(self):
    return self.datos

  def set_datos(self, datos):
    self.datos = datos

  def cargar_version_maestra(self, ruta):
    self.set_maestra(Document(ruta))

  def deconstruir(self):
    archivo = self.get_maestra()
    reactivos = archivo.tables[:]
    lista_nueva = []
    for i in range(0, len(reactivos)):
      index = 0
      archivo.save("tmp.docx")
      documento = Document("tmp.docx")
      for j in range(0, len(reactivos)):
        if j == i:
          index = 1
        else:
          documento.tables[index]._element.getparent().remove(documento.tables[index]._element)
      for paragraph in documento.paragraphs:
        p = paragraph._element
        p.getparent().remove(p)
        p._p = p._element = None
      lista_nueva.append(documento)
    return lista_nueva

  def eliminar_pilotos(self, lista):
    # Elimina los reactivos piloto para sustituirlos.
    lista_principal = lista[:]
    r31, r37, r98, r104 = 0, 0, 0, 0
    i = 1
    for doc in lista_principal:
      reactivo = doc.tables[0].cell(0,0).text
      if reactivo == "31.": r31 = i
      elif reactivo == "37.": r37 = i
      elif reactivo == "98.": r98 = i
      elif reactivo == "104.": r104 = i
      i += 1
    sin_pilotos = lista_principal[:r31] + lista_principal[r37:r98] + lista_principal[r104:]
    return sin_pilotos

  def reenumerar(self):
    copia = self.get_orden2()
    orden2 = []
    for i in copia:
      orden2.append(i)
    expression = re.compile(r"^\d*[.]")
    orden = [
          [1,5], [2,6], [3,7], [4,8], [5,1], [6,2], [7,3], [8,4], [9,9], [10,14], [11,15], [12,16], [13,17], [14,10], [15,11], [16,12], [17,13], [18,21], [19,22], [20,23], [21,18], [22,19], [23,20], [24,28], [25,29], [26,30], [27,31], [28,24], [29,25], [30,26], [31,27], [32,32], [33,33], [34,34], [35,35], [36,36], [37,37], [38,41], [39,42], [40,43], [41,38], [42,39], [43,40], [44,47], [45,48], [46,49], [47,44], [48,45], [49,46], [50,53], [51,54], [52,55], [53,50], [54,51], [55,52], [56,59], [57,60], [58,61], [59,56], [60,57], [61,58], [62,64], [63,63], [64,62], [65,67], [66,66], [67,65], [68,72], [69,73], [70,74], [71,75], [72,68], [73,69], [74,70], [75,71], [76,80], [77,81], [78,82], [79,83], [80,76], [81,77], [82,78], [83,79], [84,87], [85,88], [86,89], [87,84], [88,85], [89,86], [90,90], [91,95], [92,96], [93,97], [94,98], [95,91], [96,92], [97,93], [98,94], [99,99], [100,100], [101,101], [102,102], [103,103], [104,104], [105,109], [106,110], [107,111], [108,112], [109,105], [110,106], [111,107], [112,108], [113,117], [114,118], [115,119], [116,120], [117,113], [118,114], [119,115], [120,116], [121,125], [122,126], [123,127], [124,128], [125,121], [126,122], [127,123], [128,124], [129,133], [130,134], [131,135], [132,136], [133,129], [134,130], [135,131], [136,132], [137,141], [138,142], [139,143], [140,144], [141,137], [142,138], [143,139], [144,140], [145,147], [146,146], [147,145], [148,149], [149,148], [150,150], [151,151], [152,152], [153,153], [154,154], [155,155], [156,156], [157,157], [158,158], [159,159], [160,160], [161,161], [162,162], [163,163], [164,164], [165,165], [166,166], [167,167], [168,168], [169,169], [170,170], [171,171], [172,172], [173,173], [174,174],
        ]
    j = 0
    for i in range(0,len(orden2)-1):
      runs = orden2[i].tables[0].cell(0, 0).paragraphs[0].runs
      txt = orden2[i].tables[0].cell(0, 0).paragraphs[0].text
      if expression.match(txt):
        for r in runs:
          r.text = ""
        runs[0].text = str(orden[j][1]) + "."
        j += 1
    return orden2

  def primer_reactivo(self, docs):
    expression = re.compile(r"^\d*[.]")
    menor = 1000
    index_menor = 0
    for i in range(0, len(docs)):
      if expression.match(docs[i].tables[0].cell(0,0).text):
        num_reactivo = int( re.search(r'\d*', docs[i].tables[0].cell(0,0).text).group() )
        if num_reactivo < menor:
          menor = num_reactivo
          index_menor = i
    return docs.pop(index_menor)

  def organizar(self, docs):
    expression = re.compile(r"^\d*[.]")
    lista_ordenada = docs[:]
    nueva_lista = []
    for i in range(0, len(docs)):
      if not expression.match(docs[i].tables[0].cell(0,0).text):
        nueva_lista.append(docs[i])
      else:
        p = self.primer_reactivo(lista_ordenada)
        nueva_lista.append(p)
    return nueva_lista

  def crear_variante(self, docs, piloto1, piloto2, control1, control2):
    lista_principal = docs[:]
    r31, r98, r9, r90 = 0, 0, 0, 0
    i = 0
    for doc in lista_principal:
      reactivo = doc.tables[0].cell(0,0).paragraphs[0].text
      if reactivo == "31.": r31 = i
      elif reactivo == "98.": r98 = i
      elif reactivo == "9.": r9 = i
      elif reactivo == "90.": r90 = i
      i += 1
    armado = []
    for i in range(0,len(docs)):
      if lista_principal[i].tables[0].cell(0,0).paragraphs[0].text != "9." and lista_principal[i].tables[0].cell(0,0).paragraphs[0].text != "90.":
        armado.append(lista_principal[i])
      if i == r31: armado.append(piloto1)
      elif i == r98: armado.append(piloto2)
      elif i == r9: armado.append(control1)
      elif i == r90: armado.append(control2)
    return armado

  def generar_orden1(self):
    self.set_orden1(self.orden1)

  def generar_orden2(self):
    reenumerado = self.reenumerar()
    ordenado = self.organizar(reenumerado[:])
    self.set_orden2(ordenado)

  def ajustar_parrafos(self, doc):
    style = doc.styles['Normal']
    style.font.name = 'Arial'
    style.paragraph_format.space_after = Pt(0)
    style.paragraph_format.line_spacing_rule = WD_LINE_SPACING.SINGLE
    for t in doc.tables:
      for p in t.cell(0,0).paragraphs:
        for r in p.runs:
          if r.font.superscript:
            r.font.size = Pt(12)

  def cargar_datos(self, ord1, ord2):
    i = 0
    self.cuadernillos = []
    for d in self.get_datos():
      print(i, end=": ")
      i += 1
      print(d)
      if i != 0:
        nuevo = Exani1()
        nuevo.set_numero_cuadernillo(d[4])
        nuevo.set_bloques_piloto(d[6], d[7])
        nuevo.set_orden(d[5])
        nuevo.set_clave_control(d[9])
        nuevo.set_ccf(d[8])
        nuevo.set_version_ingles(d[13])
        nuevo.set_controles([d[10], d[11]])
        if d[5] == '1':
          nuevo.set_reactivos(ord1)
        elif d[5] == '2':
          nuevo.set_reactivos(ord2)
        ensamble.cuadernillos.append(nuevo)

  def combinar_documentos(self, files, output):
    new_document = Document()
    new_document.save("2do Ord.docx")
    master = Document("2do Ord.docx")
    composer = Composer(master)
    titulo1 = master.add_paragraph()
    titulo1.add_run("Pensamiento científico").bold = True
    titulo1.alignment = 1
    for d in files:
      f = d
      interline = master.add_paragraph()
      interline.paragraph_format.space_before = Pt(0)
      interline.paragraph_format.space_after = Pt(0)
      composer.append(f)
    for section in master.sections:
      section.left_margin = Cm(3)
      section.right_margin = Cm(3)
      section.top_margin = Cm(3)
      section.bottom_margin = Cm(3)
    self.ajustar_parrafos(master)
    master.save(output)


## Ejecución

In [ ]:

path_pilotos = '/content/drive/MyDrive/bloques_piloto_exani1/'

# @title ### Inserte los datos antes de ejecutar la celda { display-mode: "form" }
# @markdown ### Favor de asegurarse de haber ejecutado todas las celdas anteriores y de haber subido los datos csv y la versión maestra.

# @markdown #### A continuación coloque la ruta del archivo csv con los datos.
# @markdown #### Ruta del archivo CSV:
ruta_csv = "/content/drive/MyDrive/ceneval/Ecuté/Exani-1/EX1_cuadernillos.csv" # @param {type:"string"}
ruta_maestra = "/content/drive/MyDrive/ceneval/Ecuté/Exani-1/V205.docx" # @param {type:"string"}

# @markdown ---
# @markdown ## Importante:
# @markdown #### Los archivos generados serán almacenados dentro de `/content/drive/MyDrive/outputs/Exani1/`

ruta_datos_csv = ruta_csv
ruta_version_maestra = ruta_maestra
ensamble = Ensamblador(ruta_datos_csv)
ensamble.cargar_version_maestra(ruta_version_maestra)
ensamble.set_orden1(ensamble.deconstruir()[:])
ensamble.set_orden2(ensamble.deconstruir()[:])

ensamble.generar_orden1()
ensamble.generar_orden2()

ord1 = ensamble.eliminar_pilotos(ensamble.get_orden1())
ord2 = ensamble.eliminar_pilotos(ensamble.get_orden2())

ensamble.cargar_datos(ord1, ord2)

cuadernillos = ensamble.get_cuadernillos()

def posicion_reactivo(doc, numero):
  posicion = 0
  for i in doc:
    try:
      num_i = int(re.search(r'\d*', i.tables[0].cell(0,0).text).group())
    except:
      num_i = 0
    if num_i == numero:
      return posicion
    posicion += 1
  return 0

def econtrar_texto(doc, txt):
  for p in doc.paragraphs:
    if p.text == txt:
      return True
  return False

def econtrar_texto_lista(docs, txt):
  for d in docs:
    for p in d.paragraphs:
      if p.text == txt:
        return True
  return False

for c in range(1, len(cuadernillos)):
  piloto1 = cuadernillos[c].get_bloques_piloto()[0]
  piloto2 = cuadernillos[c].get_bloques_piloto()[1]

  area_piloto1 = re.search(r'[a-zA-Z]+', piloto1).group()
  area_piloto2 = re.search(r'[a-zA-Z]+', piloto2).group()

  areas = ["Pensamiento científico", "Comprensión lectora", "Redacción indirecta", "Pensamiento matemático", "Inglés"]

  p1 = path_pilotos
  p2 = path_pilotos
  p1 += area_piloto1 + "/" + piloto1 + ".docx"
  p2 += area_piloto2 + "/" + piloto2 + ".docx"

  c1 = path_pilotos + "Control9" + cuadernillos[c].get_controles()[0] + ".docx"
  c2 = path_pilotos + "Control90" + cuadernillos[c].get_controles()[1] + ".docx"

  doc_p1 = Document(p1)
  doc_p2 = Document(p2)

  control1 = Document(c1)
  control2 = Document(c2)

  # Crear un estilo para títulos
  style_titulos = doc_p1.styles['Normal']
  style_titulos.font.name = "Arial"
  style_titulos.font.size = 11
  style_titulos.font.bold = True

  if area_piloto1 != 'CLE' and not econtrar_texto(doc_p2, areas[1]):
    doc_p1.add_page_break()
    t = doc_p1.add_paragraph()
    t.add_run(areas[1]).bold = True
    t.alignment = 1
    t.style = style_titulos
    doc_p1.add_paragraph()
    doc_p1.add_paragraph()

  if area_piloto2 != 'PMA' and not econtrar_texto(doc_p2, areas[3]):
    doc_p2.add_page_break()
    t = doc_p2.add_paragraph()
    t.add_run(areas[3]).bold = True
    t.alignment = 1
    t.style = style_titulos
    doc_p2.add_paragraph()
    doc_p2.add_paragraph()

  final = ensamble.crear_variante(cuadernillos[c].get_reactivos(), doc_p1, doc_p2, control1, control2)

  #Agregar títulos
  if not econtrar_texto_lista(final, areas[2]):
    final[posicion_reactivo(final, 67)].add_page_break()
    t = final[posicion_reactivo(final, 67)].add_paragraph()
    t.add_run(areas[2]).bold = True
    t.alignment = 1
    t.style = style_titulos
    final[posicion_reactivo(final, 67)].add_paragraph()
    final[posicion_reactivo(final, 67)].add_paragraph()

  if not econtrar_texto_lista(final, areas[4]):
    final[posicion_reactivo(final, 144)].add_page_break()
    t = final[posicion_reactivo(final, 144)].add_paragraph()
    t.add_run(areas[4]).bold = True
    t.alignment = 1
    t.style = style_titulos
    final[posicion_reactivo(final, 144)].add_paragraph()
    final[posicion_reactivo(final, 144)].add_paragraph()

  if area_piloto1 == 'CLE' and not econtrar_texto_lista(final, areas[1]):
    final[posicion_reactivo(final, 31)].add_page_break()
    t = final[posicion_reactivo(final, 31)].add_paragraph()
    t.add_run(areas[1]).bold = True
    t.alignment = 1
    t.style = style_titulos
    final[posicion_reactivo(final, 31)].add_paragraph()
    final[posicion_reactivo(final, 31)].add_paragraph()

  if area_piloto2 == 'PMA' and not econtrar_texto_lista(final, areas[3]):
    final[posicion_reactivo(final, 98)].add_page_break()
    t = final[posicion_reactivo(final, 98)].add_paragraph()
    t.add_run(areas[3]).bold = True
    t.alignment = 1
    t.style = style_titulos
    final[posicion_reactivo(final, 98)].add_paragraph()
    final[posicion_reactivo(final, 98)].add_paragraph()

  t = None
  try:
    os.mkdir("/content/drive/MyDrive/outputs/")
  except:
    try:
      os.mkdir("/content/drive/MyDrive/outputs/Exani1/")
    except:
      print("Existentes")
  finally:
    print("Carpetas creadas")


  ensamble.combinar_documentos(final, "/content/drive/MyDrive/outputs/Exani1/" + cuadernillos[c].get_numero_cuadernillo() + ".docx")
  print(cuadernillos[c].get_numero_cuadernillo() + ".docx" + ": Saved")


0: ['Num', 'Tipo', 'Version maestra', 'Periodo de aplicacion ', 'Cuadernillos', 'Ord', 'Bloque piloto 1', 'Bloque piloto 2', 'CCF', 'Clave control', 'Control1', 'Control2', 'Descripcion', 'Version de Inglos', 'Anio']
1: ['23', 'Nacional ', 'V205', '', '12523', '1', 'PC705', 'RIN705', 'EX1IN-12523-37755', 'O', 'C', 'A', 'Nacional 205 2024 C_12523 1 Ord 001', '10', '2024']
2: ['24', 'Nacional ', 'V205', '', '12524', '2', 'PC706', 'RIN706', 'EX1IN-12524-37756', 'P', 'A', 'B', 'Nacional 205 2024 C_12524 2 Ord 002', '10', '2024']
3: ['25', 'Nacional ', 'V205', '', '12525', '1', 'PC707', 'RIN707', 'EX1IN-12525-37757', 'Q', 'B', 'C', 'Nacional 205 2024 C_12525 1 Ord 003', '10', '2024']
4: ['26', 'Nacional ', 'V205', '', '12526', '2', 'PC708', 'RIN708', 'EX1IN-12526-37758', 'R', 'C', 'A', 'Nacional 205 2024 C_12526 2 Ord 004', '10', '2024']
Carpetas creadas
12523.docx: Saved
Existentes
Carpetas creadas
12524.docx: Saved
Existentes
Carpetas creadas
12525.docx: Saved
Existentes
Carpetas creadas
